In [1]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/home/onyxia/work/cine-insights/src/models')
sys.path.append('/home/onyxia/work/cine-insights/src/data')
sys.path.append('/home/onyxia/work/cine-insights/src/features')
sys.path.append('/home/onyxia/work/cine-insights/src/models')

sys.path.append('/home/onyxia/work/cine-insights/models')


In [3]:
import pandas as pd
import joblib
import numpy as np
python_interpretor="/home/onyxia/work/TMDb-Box-Office-Prediction/Codes/BOF_ENv/bin/python"

# import_data.py

In [5]:
import Import_data
from  Import_data import *

In [6]:
filename="Raw_df_50_ligne.csv"
path_dir="/home/onyxia/work/cine-insights/src/data/"

In [7]:
df=Import_data.load_data(path_dir+filename,"Raw_Data")

Raw_Data: shape is (50, 25)


In [8]:
df.head(2)

,Unnamed: 0,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,Keywords
0,0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."


# Preporcess_TrainData.py

In [9]:
import Preprocess_trainData
from  Preprocess_trainData import *

# sans la colonne duraion

In [10]:
(x_train,y_train),(x_test,y_test),(x_val,y_val),ct=Preprocess_trainData.preprocessing_data(df)

(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 



In [91]:
x_val

,budget,popularity,has_homepage,release_month,release_year,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,...,genre_Romance,genre_Science_Fiction,genre_Thriller,genre_War,genre_Western,spoken_languages_count,production_countries_count,production_companies_count,Keywords_count,released
45,0.360772,0.319862,1.0,9.0,1995.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,21.0,1.0
26,0.839998,0.391493,1.0,4.0,2013.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,11.0,1.0


# Avec la colonne duration

# Preprocessing_PredictedData.py

In [11]:
import Preprocessing_PredictedData
from Preprocessing_PredictedData import *

In [12]:
X=df.drop(["revenue"],axis=1)
X.columns

Index(['Unnamed: 0', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'runtime', 'adult', 'backdrop_path', 'budget',
       'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'Keywords'],
      dtype='object')

### SANS LA COLONNE Duration


In [94]:
X_transf=Preprocessing_PredictedData.preprocessing_pipeline(X,ct)

In [97]:
X_transf.columns

Index(['budget', 'popularity', 'has_homepage', 'release_month', 'release_year',
       'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       'genre_Crime', 'genre_Drama', 'genre_Family', 'genre_Fantasy',
       'genre_Mystery', 'genre_Romance', 'genre_Science_Fiction',
       'genre_Thriller', 'genre_War', 'genre_Western',
       'spoken_languages_count', 'production_countries_count',
       'production_companies_count', 'Keywords_count', 'released'],
      dtype='object')

In [15]:
set(x_test.columns).difference(set(X_transf.columns))

set()

In [16]:
X_transf.shape

(50, 24)

In [17]:
X_transf.isna().sum().sum()

0

## Avec LA COLONNE Duration


In [211]:
X_transf_with_duration.shape

(50, 25)

# TRaining

In [95]:
import TrainerS1
from TrainerS1 import *

# SANS DURATIOn

In [98]:
model1=TrainerS1.XGBRegressorWrapper(df,with_duration=False)
 


(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 162 candidates, totalling 324 fits


In [99]:
y_val_pred=model1.predict_(X)

In [102]:
y=df.revenue
X.shape,y.shape

((50, 24), (50,))

In [104]:
evaluation_data = pd.DataFrame({'y_true': y, 'y_pred': y_val_pred})


In [34]:
model1.param_Combination

[{'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.7},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.8},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.9},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'objective': 'reg:squarederror',
  'subsample': 0.7},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'objective': 'reg:squarederror',
  'subsample': 0.8},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'object

In [21]:
model1.display_results()

,Best parameters,Training RMSLE,Test RMSLE,Test MAPE,Training time
0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",-0.135773,0.797797,63.959912,6.0818


In [22]:
X=df.drop(["revenue"],axis=1)
X.columns

Index(['Unnamed: 0', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'runtime', 'adult', 'backdrop_path', 'budget',
       'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'Keywords'],
      dtype='object')

In [23]:
model1.predict_(X).shape

(50,)

# Avec Duration

In [176]:
model2.display_results()

,Best parameters,Training RMSLE,Test RMSLE,Test MAPE,Training time
0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",-0.141859,0.897502,57.725694,4.4785


In [177]:
best_param2=model2.best_params_
best_param2

{'colsample_bytree': 0.7,
 'learning_rate': 0.03,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 500,
 'objective': 'reg:squarederror',
 'subsample': 0.9}

In [185]:
X2_t=Preprocessing_PredictedData.preprocessing_pipeline(X,ct,with_duration=False)

In [186]:
X2_t.shape

(50, 24)

In [187]:
X2_t.columns

Index(['budget', 'popularity', 'has_homepage', 'release_month', 'release_year',
       'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       'genre_Crime', 'genre_Drama', 'genre_Family', 'genre_Fantasy',
       'genre_Mystery', 'genre_Romance', 'genre_Science_Fiction',
       'genre_Thriller', 'genre_War', 'genre_Western',
       'spoken_languages_count', 'production_countries_count',
       'production_companies_count', 'Keywords_count', 'released'],
      dtype='object')

In [190]:
model1.predict_(X).shape

(50,)

In [158]:
X2=df.drop(["revenue"],axis=1)
X2.columns

Index(['Unnamed: 0', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'runtime', 'adult', 'backdrop_path', 'budget',
       'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'Keywords'],
      dtype='object')

In [ ]:
# MAIN.py

In [35]:
import Main_no_dur_variable1
from Main_no_dur_variable1 import *

In [36]:
Main_no_dur_variable1.run_xgreg()

Raw_df: shape is (50, 25)
(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 162 candidates, totalling 324 fits


,Best parameters,Training RMSLE,Test RMSLE,Test MAPE,Training time
0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",-0.135773,0.797797,63.959912,5.8013


In [28]:
get_hyperparameter_combinations=XGBRegressorWrapper2().get_hyperparameter_combinations()

In [29]:
get_hyperparameter_combinations

[{'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.7},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.8},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 500,
  'objective': 'reg:squarederror',
  'subsample': 0.9},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'objective': 'reg:squarederror',
  'subsample': 0.7},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'objective': 'reg:squarederror',
  'subsample': 0.8},
 {'colsample_bytree': 0.7,
  'learning_rate': 0.03,
  'max_depth': 5,
  'min_child_weight': 3,
  'n_estimators': 1000,
  'object

In [43]:
import Test_mlflow_log5 as mlog

In [44]:
mlog.mlflow_fun(model1,"Experiment1:save only paramater",1)

In [54]:
import Test_mlflow_log57 as mlog2

In [55]:
mlog2.mlflow_fun(model1,"Experiment2:save only paramater and metrics",2)

In [65]:
import Test_mlflow_log62 as mlog3

In [57]:
mlog3.mlflow_fun(model1,"Experiment2:save only paramater and metrics",2)

2024/04/29 14:43:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:24 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:28 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:32 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:36 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:40 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:45 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:48 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/04/29 14:43:52 WARNING mlflow.sklearn: Model was missing fu

In [87]:
import Trainer_S13
from Trainer_S13 import *

In [106]:
model1=Trainer_S13.XGBRegressorWrapper(df,with_duration=False)


(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 3 candidates, totalling 6 fits


In [118]:
import logging

# Définissez le niveau de journalisation sur DEBUG pour voir la trace complète de l'erreur lors de l'inférence des exigences pip
logging.getLogger().setLevel(logging.DEBUG)


In [128]:
import Test_mlflow_log614 as mlog4

In [126]:
evaluation_data=pd.read_csv("evaluation_data.csv")

In [129]:
mlog4.mlflow_fun(model1,"Experiment1: Add data",7,evaluation_data=evaluation_data)

2024/04/29 18:57:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp4rt6akuz/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.2', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/04/29 18:57:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9xsjt8wr/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.2', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/04/29 18:57:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpflwnp46g/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.2', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


In [135]:
import mlfow_log3 as mlog5
mlog5.mlflow_fun(model1,"Experiment1: Add data good1",7,evaluation_data=X)

In [136]:
mlog5.mlflow_fun(model1,"Experiment1: Add data good1",7,evaluation_data=X)

TypeError: Trainer_S13.XGBRegressorWrapper._train_model() argument after ** must be a mapping, not builtin_function_or_method

In [62]:
mlog3.mlflow_fun(model1,"Experiment2:save all meta_data ",1)

2024/04/29 14:59:41 INFO mlflow.tracking.fluent: Experiment with name 'Experiment2:save all meta_data' does not exist. Creating a new experiment.


In [63]:
import Trainer_Sa
from Trainer_Sa import *

In [64]:
model4=Trainer_Sa.XGBRegressorWrapper(df,with_duration=False)


(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 162 candidates, totalling 324 fits


In [66]:

mlog3.mlflow_fun(model4,"Experiment2:save all meta_data on param_combi set",4)

2024/04/29 15:09:26 INFO mlflow.tracking.fluent: Experiment with name 'Experiment2:save all meta_data on param_combi set' does not exist. Creating a new experiment.


In [69]:
import Main_no_dur_variable1a
from Main_no_dur_variable1a import *

In [68]:
Main_no_dur_variable1.run_xgreg()

Raw_df: shape is (50, 25)
(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 162 candidates, totalling 324 fits


,Best parameters,Training RMSLE,Test RMSLE,Test MAPE,Training time
0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",-0.135773,0.797797,63.959912,9.0426


In [84]:
import Evaluation6
from Evaluation6 import *

In [85]:
experiment_name="Experiment2:save all meta_data"
evaluator=Evaluation6.get_best_model(experiment_name)

Best run:
run_id                                            c3b9cdd87f4a4c07bed6ff9245e2d3f9
experiment_id                                                                    5
status                                                                    FINISHED
artifact_uri                     mlflow-artifacts:/5/c3b9cdd87f4a4c07bed6ff9245...
start_time                                        2024-04-29 14:59:50.290000+00:00
end_time                                          2024-04-29 14:59:54.616000+00:00
metrics.Test RMSLE                                                        0.870886
metrics.Training time                                                       3.0289
metrics.Test MAPE                                                         56.52931
metrics.Training RMSLE                                                   -0.161327
params.objective                                                  reg:squarederror
params.subsample                                                             

In [86]:
evaluator[0]

params.objective           reg:squarederror
params.subsample                        0.7
params.n_estimators                     500
params.learning_rate                   0.03
params.min_child_weight                   3
params.num_appli                          1
params.colsample_bytree                 0.7
params.max_depth                          7
Name: 0, dtype: object

In [165]:
import Final_Trainer80
from Final_Trainer80 import *
model1=Final_Trainer80.XGBRegressorWrapper(df,with_duration=False)


(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 



In [166]:
import final_mlflow_log9


In [167]:
final_mlflow_log9.mlflow_fun(model1,"NewExperiment1:Final Add data good 2",10,evaluation_data=X)

In [ ]:
git checkout -b dev_models origin/dev_models
